In [ ]:
'''
The following variables are defined: "instrument" for the instrument to trade (e.g. NIFTY/BANKNIFTY), 
"entry_time" and "exit_time" for the start and end times of the trading period 
(e.g. 09:30 and 15:10, respectively), "price_value" for the price at which the option will be sold, 
"entry_percentage" for the percentage of the option's price at which the algorithm will trigger the sale (e.g. 80%), 
"sl_percentage" for the percentage of the option's price at which a stop-loss will be triggered (e.g. 10%), 
"api_key" for the API key to use, and "access_token" for the access token to use.
'''
instrument='NIFTY/BANKNIFTY'
entry_time='09:30'
exit_time ='15:10'
price_value=100                                             #Options price to sell
entry_percentage=0.8                                        #algo will sell the option once the options price reaches 80
sl_percentage=0.1 #10% SL
api_key = 'api_key_here'
access_token = 'access_token_here

In [ ]:
from kiteconnect import KiteConnect, KiteTicker
import datetime as dt
import pandas as pd
import time

class OptionsSelling():
    def __init__(self, instrument,entry_time,exit_time,price_value,entry_percentage, sl_percentage, api_key, access_token):
        self.instrument = instrument
        self.entry_time = dt.datetime.strptime(entry_time, "%H:%M").time()
        self.exit_time = dt.datetime.strptime(exit_time, "%H:%M").time()
        self.entry_percentage = entry_percentage
        self.price_value = price_value
        self.sl = sl_percentage
        self.kite =  KiteConnect(api_key=api_key)
        self.kite.set_access_token(access_token)
        self.kws =KiteTicker(api_key = api_key, access_token = access_token) 
        self.instrument_list = pd.DataFrame(self.kite.instruments())
        df = self.instrument_list[(self.instrument_list['name']==self.instrument) &(self.instrument_list['segment']=='NFO-OPT')]
        expiry = df.expiry.min()
        self.df = df[df['expiry']==expiry]
        self.subscribed_tokens=self.get_subscribed_tokens()
        self.tick_data={}
        self.kws.on_ticks = self.on_ticks
        self.kws.on_connect = self.on_connect
        self.kws.connect(threaded=True)
        self.execute_algo()
        
    def get_exchange_symbol(self):
        exchange_symbol = 'NSE:NIFTY 50' if self.instrument == 'NIFTY' else 'NSE:NIFTY BANK'
        return exchange_symbol

    def get_subscribed_tokens(self):
        subscribed_tokens = {}
        for index, row in self.df.iterrows():
            subscribed_tokens[row['instrument_token']]=row['tradingsymbol']
        return subscribed_tokens
#     def jugaad_login(self):
#         kite1 = jugaad_zerodha()
#         kite1.user_id=creds['user_id']
#         kite1.password = creds['password']
#         try:
#             json_res = kite1.login_step1()
#             twofa = pyotp.TOTP(creds['totp_key']).now()
#             kite1.twofa=twofa
#             json_res_1 = kite1.login_step2(json_res)
#             kite1.enc_token = kite1.r.cookies['enctoken']
#         except Exception as e:
#             print(f'Error logging in to Jugaad Zerodha: {e}')
#             raise
#         return kite1
    def on_ticks(self, ws, ticks):
        for tick in ticks:
            instrument_token = tick['instrument_token']
            last_price = tick['last_price']
            timestamp = tick['timestamp']
            if instrument_token not in self.tick_data:
                self.tick_data[instrument_token] = []
            self.tick_data[instrument_token].append({'timestamp': timestamp, 'ltp':last_price})

    def on_connect(self, ws, response):
        tokens = [int(x) for x in list(self.subscribed_tokens.keys())]
        ws.subscribe(tokens)
        ws.set_mode(ws.MODE_FULL,tokens)  
        
    def place_order(self,symbol, transaction_type,qty, trigger_price):
        order_id = self.kite.place_order(tradingsymbol=symbol,
                                         exchange=self.kite.EXCHANGE_NFO,
                                         transaction_type=transaction_type,
                                         quantity=qty,
                                         order_type=kite.ORDER_TYPE_SL,
                                         product=kite.PRODUCT_MIS,
                                         trigger_price= trigger_price,
                                         variety = kite.VARIETY_REGULAR)
        return order_id

    def get_closest_options(self):
        for key in self.subscribed_tokens.keys():
            self.subscribed_tokens[key] = {'tradingsymbol': self.subscribed_tokens[key], 'ltp': self.tick_data[key][-1]['ltp']}
        test = pd.DataFrame(self.subscribed_tokens).T
        test
        closest_ce = None
        closest_pe = None
        smallest_ce_difference = None
        smallest_pe_difference = None
        for index, row in test.iterrows():
            difference = abs(row['ltp'] - self.price_value)
            if 'CE' in row['tradingsymbol']:
                if smallest_ce_difference is None or difference < smallest_ce_difference:
                    smallest_ce_difference = difference
                    closest_ce = row
            elif 'PE' in row['tradingsymbol']:
                if smallest_pe_difference is None or difference < smallest_pe_difference:
                    smallest_pe_difference = difference
                    closest_pe = row
        return {'ce_token':closest_ce.name, 'ce_symbol':closest_ce.tradingsymbol, 'ce_ltp':closest_ce.ltp,'pe_token':closest_pe.name, 'pe_symbol':closest_pe.tradingsymbol, 'pe_ltp':closest_pe.ltp }
    def square_off(self):
        orders = pd.DataFrame(self.kite.orders())
        orders = orders[(orders['status']=='OPEN')]
        if not orders.empty:
            for index, row in orders.iterrows():
                cancelled_order_id =self.kite.cancel_order('regular', row['order_id'])
        for index, row in pd.DataFrame(self.kite.positions()['net']).iterrows():
            transaction_type = 'SELL' if row['quantity'] > 0 else 'BUY' if row['quantity'] < 0 else None
            if row['quantity']!=0:
                order_id = self.kite.place_order(tradingsymbol = row['tradingsymbol'],
                                                 exchange  = row['exchange'],
                                                 transaction_type = transaction_type,
                                                 quantity = row['quantity'],
                                                 order_type = "MARKET",
                                                 variety = "regular",
                                                 product = row['product'])
    def execute_algo(self):
        current_time = dt.datetime.now().time()
        entry_time=self.entry_time
        if current_time < entry_time:
            time_difference = dt.datetime.combine(dt.date.today(), entry_time) - dt.datetime.combine(dt.date.today(), current_time)
            sleep_time = time_difference.total_seconds()
            print(f'sleeping for {sleep_time} seconds')
            time.sleep(sleep_time)
        print(f'checking for closest {self.price_value} bucks options @{dt.datetime.now().time()}')
        data = self.get_closest_options()
        print(f'Nearest {self.price_value} bucks options @{dt.datetime.now().time()}')
        current_time = dt.datetime.now().time()
        market_close = self.exit_time
        ce_flag=True
        pe_flag=True
        tick_data = self.tick_data
        print('Monitoring for trades')
        while(current_time< market_close and (ce_flag or pe_flag)):
            ce_ltp = tick_data[data['ce_token']][-1]['ltp']
            pe_ltp = tick_data[data['pe_token']][-1]['ltp']
            print(f"CE {data['ce_symbol']} LTP {ce_ltp}")
            time.sleep(1)
            print(f"PE {data['pe_symbol']} LTP {pe_ltp}")
            if ce_ltp <=self.entry_percentage*data['ce_ltp'] and ce_flag:

                print(f"Order placed for {data['ce_symbol']} @ {ce_ltp} and SL is {ce_ltp*(1-self.sl)} and time is {current_time}")
#                 self.place_order(data[ce_symbol], self.kite.TRANSACTION_TYPE_BUY,self.qty, ce_ltp*(1-self.sl))
                ce_flag=False
            if pe_ltp <=self.entry_percentage*data['pe_ltp'] and pe_flag:
#                 self.place_order(data[pe_symbol], self.kite.TRANSACTION_TYPE_SELL,self.qty, pe_ltp*(1+self.sl))
                print(f"Order placed for {data['ce_symbol']} @ {pe_ltp} and SL is {pe_ltp*(1+self.sl)} and time is {current_time}")
                pe_flag=False
            current_time = dt.datetime.now().time()
#         self.square_off()
#         print('exiting....ciao')

if __name__ == "__main__":
    premium_based_algo = OptionsSelling(instrument,entry_time,exit_time,price_value,entry_percentage, sl_percentage, api_key, access_token)